In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import os

In [2]:
conv = pd.read_csv('../data/ecuador_earthquake_2016/es/conversations.csv', parse_dates=['timestamp'],dtype={'id':object, 'conversation_id':object,'in_reply_to_status_id':object})
conv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94776 entries, 0 to 94775
Data columns (total 13 columns):
id                       94776 non-null object
screen_name              94776 non-null object
latitude                 187 non-null float64
longitude                187 non-null float64
lang                     94776 non-null object
in_reply_to_status_id    50747 non-null object
text                     94776 non-null object
timestamp                94765 non-null datetime64[ns]
conversation_id          94776 non-null object
conversation_deep        94776 non-null int64
num_replies              94776 non-null int64
num_users                94776 non-null int64
url                      94776 non-null object
dtypes: datetime64[ns](1), float64(2), int64(3), object(7)
memory usage: 9.4+ MB


## sample randomly for roots (original) tweets

In [4]:
conv = conv[(conv.num_replies>2) & (conv.num_replies<20)]
conv.shape

(14851, 22)

In [5]:
conv = conv[conv.num_users>1]
conv.shape

(14674, 22)

In [6]:
conv['label'] = ''

In [8]:
roots = conv[conv.in_reply_to_status_id.isnull()].copy()
roots.shape

(2393, 23)

In [9]:
rows_filter=roots.text.str.contains('oraci')
roots.loc[rows_filter, 'label'] = 'emotional'
roots[rows_filter].shape

(41, 23)

In [10]:
roots[roots.label=='emotional'].shape

(41, 23)

In [11]:
rows_filter=roots.text.str.contains('ofre')
roots.loc[rows_filter, 'label'] = 'offer'
roots[rows_filter].shape

(6, 23)

In [12]:
rows_filter=roots.text.str.contains('muertos')
roots.loc[rows_filter, 'label'] = 'affected'
roots[rows_filter].shape

(164, 23)

In [13]:
rows_filter=roots.text.str.contains('daños')
roots.loc[rows_filter, 'label'] = 'infrastructure'
roots[rows_filter].shape

(48, 23)

In [17]:
rows_filter=roots.text.str.contains('ayuda')
roots.loc[rows_filter, 'label'] = 'need'
roots[rows_filter].sample(10,random_state=3).text.values

array(['Hermanos de Esmeraldas y Manabi necesitan ayuda agua, medicina, colchones, alimentos favor canalizar a través de Cruz Roja',
       '#SismoEcuador Venezuela enviará a Ecuador un avión con ayuda tras terremoto https://t.co/mSscvNzcW0 Q puede enviar? https://t.co/nQPMNPaY4y',
       'Mi mamá ya está en un refugio en Bahia. Mis tías perdieron sus casas como muchos más. Hay gente desaparecida y la ayuda urge. #pormanabi',
       '#SismoEcuador se traslada hospital móvil a hacia Pedernales para asistir a damnificados y llama a voluntarios que quieran prestar ayuda',
       'Ya me voy, buenas madrugadas, todo va a estar bien con la ayuda de Dios, ecuador:)',
       '#Venezuela fue el primerito en enviar ayuda a #Ecuador comida y medicinas, vamos a necesitar un temblor para ver si nos dan las ayuda aquí',
       '¿De donde sacaran la ayuda? De los galpones de comida que tiene acaparado el gobierno para comprar VOTOS.  https://t.co/ffx9DddzZJ',
       'Aun no llegan ayuda a #Pedernales

In [ ]:
rows_filter=roots.text.str.contains('desapareci')
roots.loc[rows_filter, 'label'] = 'missing'
roots[rows_filter].shape

In [ ]:
rows_filter=roots.text.str.contains('evacua')
roots.loc[rows_filter, 'label'] = 'evacuation'
roots[rows_filter].shape

In [ ]:
rows_filter=roots.text.str.contains('precaucion')
roots.loc[rows_filter, 'label'] = 'precaution'
roots[rows_filter].shape

In [ ]:
sample = roots[roots.label!='']
sample.shape

In [ ]:
others = roots[roots.label==''].sample(300, random_state=1)
others.shape

In [ ]:
sample = sample.append(others, ignore_index=True)
sample.shape

In [ ]:
#sample.to_csv('../data/ecuador_earthquake_2016/conversations_es_sampling_keywords.csv', index=False)

# conversations

In [5]:
conv = pd.read_csv('../data/ecuador_earthquake_2016/es/conversations.csv', parse_dates=['timestamp'], dtype={'id':object, 'conversation_id':object,'in_reply_to_status_id':object})
conv.shape

(94776, 22)

In [6]:
annotated = pd.read_csv('../data/ecuador_earthquake_2016/es/conversations_annotated.csv', parse_dates=['timestamp'], dtype={'id':object,'conversation_id':object, 'in_reply_to_status_id':object})
annotated.shape

(2629, 60)

In [7]:
conv_ds = conv[(conv.num_replies>2) & (conv.num_replies<20)]
conv_ds.shape

(14851, 22)

In [8]:
conv_ds = conv_ds[conv_ds.num_users>1]
conv_ds.shape

(14674, 22)

In [9]:
conv_ds = conv_ds[conv_ds.conversation_id.isin(annotated.id)]
conv_ds.shape

(2872, 22)

In [ ]:
#conv_ds.to_csv('../data/ecuador_earthquake_2016/es/conversations_sample.csv', index=False)

# dialogs 

In [3]:
def build_dialogs_sample(input_file, output_file, min_turns=5, max_turns=10):
    dialogs = pd.read_csv(
        input_file, parse_dates=['timestamp'],
        dtype={'id':object, 'conversation_id':object,'in_reply_to_status_id':object})
    print(dialogs.shape)
    dfilter = dialogs[dialogs.in_reply_to_status_id.isnull()]
    print(dfilter.shape)
    dfilter = dfilter[(dfilter.timestamp >= datetime(2016,4,16)) & (dfilter.timestamp < datetime(2016,5,16))]
    print(dfilter.shape)
    dfilter = dfilter[(dfilter.turns>=min_turns) & (dfilter.turns<=max_turns)]
    print(dfilter.shape)
    dialogs2 = dialogs[dialogs.dialog_id.isin(dfilter.dialog_id)]
    print(dialogs2.shape)
    dialogs2.to_csv(output_file, index=False)

In [5]:
build_dialogs_sample(
    '../data/ecuador_earthquake_2016/es/dialogs.csv',
    '../data/ecuador_earthquake_2016/es/dialogs_sample.csv',
    min_turns=6,
    max_turns=10
)

(11524, 15)
(1415, 15)
(1037, 15)
(544, 15)
(3839, 15)


In [9]:
build_dialogs_sample(
    '../data/ecuador_earthquake_2016/en/dialogs.csv',
    '../data/ecuador_earthquake_2016/en/dialogs_sample.csv',
    min_turns=3,
    max_turns=10
)

(10870, 15)
(1243, 15)
(197, 15)
(172, 15)
(1159, 15)
